In [1]:
#импортируем библиотеки

import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings; warnings.filterwarnings(action='ignore')


from sklearn.impute import KNNImputer
from plotly import graph_objects as go
from folium import Map, Choropleth
from folium import Map, Marker # импортируем карту и маркер
from folium.plugins import MarkerCluster # импортируем кластер
from folium import Map, Choropleth # импортируем карту и хороплет

In [2]:
import matplotlib
%config InlineBackend.figure_format = 'retina'  #для более красивых графиков. Нужно один раз запустить в начале проекта
matplotlib.style.use('ggplot')
plt.rc('figure', figsize=(10, 6))

In [3]:
# снимаем ограничение на количество столбцов
pd.set_option('display.max_columns', None)

# снимаем ограничение на ширину столбцов
pd.set_option('display.max_colwidth', None)

# игнорируем предупреждения
pd.set_option('chained_assignment', None)

# выставляем ограничение на показ знаков после запятой
pd.options.display.float_format = '{:,.2f}'.format

# устанавливаем стиль графиков
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":200, 'savefig.dpi':300})
sns.set_context('notebook')  
sns.set_style("ticks")

In [4]:
#Прочитаем датайфрейм:

try:
    df = pd.read_csv('/datasets/ecom_dataset_upd.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/ecom_dataset_upd.csv')

In [14]:
# напишем функцию для обзора данных:

def df_review(df):
    display(df.head(10))
    print('---------------------------------------------------------------------------------------------------------')
    print(df.info())
    print('---------------------------------------------------------------------------------------------------------')
    print(df.describe())
    print('---------------------------------------------------------------------------------------------------------')
    print(df.duplicated().sum())

In [12]:
df_review(df)


,date,customer_id,order_id,product,quantity,price
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.00
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12, h25",1,194.00
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.00
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.00
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.00
5,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Шеффлера Лузеана d-9 см,1,164.00
6,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Юкка нитчатая d-12 см h-25-35 см,1,134.00
7,2018100108,375e0724-f033-4c76-b579-84969cf38ee2,68479,Настенная сушилка для белья Gimi Brio Super 100,1,824.00
8,2018100108,6644e5b4-9934-4863-9778-aaa125207701,68478,"Таз пластмассовый 21,0 л круглый ""Водолей"" С614, 1404056",1,269.00
9,2018100109,c971fb21-d54c-4134-938f-16b62ee86d3b,68480,Чехол для гладильной доски Colombo Persia Beige 130х50 см из хлопка 5379,1,674.00


---------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7474 entries, 0 to 7473
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7474 non-null   int64  
 1   customer_id  7474 non-null   object 
 2   order_id     7474 non-null   int64  
 3   product      7474 non-null   object 
 4   quantity     7474 non-null   int64  
 5   price        7474 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 350.5+ KB
None
---------------------------------------------------------------------------------------------------------
                  date   order_id  quantity     price
count         7,474.00   7,474.00  7,474.00  7,474.00
mean  2,018,912,614.03  49,449.88      2.36    478.74
std         427,887.84  32,679.64     14.50    901.62
min   2,018,100,100.00  12,624.00      1.00      9.00
25%   2

In [18]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H')

In [19]:
df['product'] = df['product'].str.lower()
df['product'] = df['product'].replace('ё', 'е', regex=True)

Данные извлечены и готовы к анализу. Пропусков и дубликатов не обнаружено. Типы данных соответствуют содержимому.

Цель исследования


Анализ и сегментация товарного ассортимента в зависимости от выручки и количества продаж:



- проведение исследовательского анализа данных,
- сегментация товарного ассортимента в зависимости от выручки и количества продаж,
- формулировка и проверка статистических гипотез.

  * гипотеза о различии среднего чека основного ассортимента и дополнительного ассортимента;
  * гипотеза о различии продаж основного и дополнительного ассортимента по будням и отдельно по выходным.

 # E-commerce - Cегментация покупателей по профилю потребления

<b> Цель: </b>

Анализ и сегментация покупателей на основе истории их покупок


<b> Описание данных: </b>

Датасет описывает транзакции интернет-магазина товаров для дома и быта «Пока все ещё тут».

Колонки:

* *date* — дата заказа;
* *customer_id* — идентификатор покупателя;
* *order_id*  — идентификатор заказа;
* *product* — наименование товара;
* *quantity*  — количество товара в заказе;
* *price* — цена товара.

<b> Задачи: </b>

1. Открыть файл с данными и изучить общую информацию.
2. Подготовить данные.
    - Проверить наличие дубликатов и пропусков.
    - Проверить данные на аномалии (оптовые заказы, наличие у одного номера заказа единственного номера покупателя).
    - Проверить типы данных. Откорректировать при необходимости.
    - Добавить столбец hour, в котором будет указан конкретный час, в который был сделан заказ.
    - Добавить столбец weekday, в котором будет указан день недели заказа.
    - Добавить столбец month, с месяцем заказа.
    - Добавить столбец year, с годом заказа, чтобы учитывать его при анализе месяца продажи.
    - Выделить товары в отдельные категории (около 5). Выделить первые слова и смотреть по ним.
    
    

3. Провести исследовательский анализ.
   - Определить количество покупателей
   - Определить количество заказов
   - Определить количество уникальных товаров
   - Определить среднюю цену одного товара
   - Определить среднее количество заказов на одного покупателя
   - Проанализировать в какие часы, дни недели и месяца происходит больше всего заказов.
   - Определить максимальную и минимальную даты. Построить гистограмму по дате и времени.


4. Поделить ассортимент на 3 группы в зависимости от выручки и количества продаж:
   - "A" - высокая выручка, большое количество продаж;
   - "B" - средняя выручка, среднее количество продаж;
   - "C" - малая выручка, малое количество продаж.
   - Основной ассортимент: "AA", "AB", "BA", "AC", "CA".
   - Дополнительный ассортимент: "BB", "BC", "CB", "CC".
   
   
5. Проверка гипотез.